In [1]:
"""
Исправленный и оптимизированный вариант кода для генерации ассоциативных правил (a) -> label.
Реализация опирается на:
  - Предобработку: дискретизацию числовых признаков и отбор ключевых столбцов.
  - Преобразование данных в транзакции с кодированием айтемов через Pandas category.
  - Построение FP-дерева с оптимизированным обновлением (с использованием счётчиков, без дублирования списков).
  - Майнинг частых наборов (FP-growth) и генерация правил, где правая часть – это Label.
  - Вывод всех найденных правил и сохранение результатов в CSV файлы.
"""

import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from itertools import chain, combinations

##############################################
# 1. Предобработка данных: дискретизация и отбор признаков
##############################################

def preprocess_data(df, selected_columns):
    """
    Выполняет предварительную обработку DataFrame:
    - Отбирает указанные столбцы.
    - Для числовых столбцов выполняет дискретизацию (пример: деление на 3 категории).
    - Преобразует значения в строковые представления вида "Column:Value".
    Возвращает DataFrame с отобранными/дискретизированными столбцами.
    """
    # Пример дискретизации для непрерывных признаков.
    if 'Flow Duration' in df.columns:
        df['Flow Duration_cat'] = pd.cut(df['Flow Duration'],
                                         bins=[0, 1e5, 1e6, float('inf')],
                                         labels=['short', 'medium', 'long'])
    if 'Flow Bytes/s' in df.columns:
        # Замена бесконечностей и NaN на 0, затем дискретизация.
        df['Flow Bytes/s'] = df['Flow Bytes/s'].replace([float('inf'), np.nan], 0)
        df['Flow Bytes/s_cat'] = pd.cut(df['Flow Bytes/s'],
                                        bins=[-0.1, 1e5, 1e6, float('inf')],
                                        labels=['low', 'mid', 'high'])
    # Формирование нового списка столбцов.
    new_cols = []
    for col in selected_columns:
        if col in ['Flow Duration', 'Flow Bytes/s']:
            if col == 'Flow Duration' and 'Flow Duration_cat' in df.columns:
                new_cols.append('Flow Duration_cat')
            elif col == 'Flow Bytes/s' and 'Flow Bytes/s_cat' in df.columns:
                new_cols.append('Flow Bytes/s_cat')
        else:
            new_cols.append(col)
    df_proc = df[new_cols].copy()
    # Преобразуем значения каждого столбца к формату "Column:Value"
    for col in df_proc.columns:
        df_proc[col] = df_proc[col].astype('category')
        df_proc[col] = df_proc[col].apply(lambda x: f"{col}:{x}")
    return df_proc
##############################################
# 2. Преобразование данных в транзакции с кодированием айтемов
##############################################

def transactions_from_df(df):
    """
    Преобразует DataFrame в список транзакций,
    где каждая транзакция – это список строк вида "Column:Value".
    """
    return df.values.tolist()

##############################################
# 3. Реализация FP-дерева (оптимизированная версия)
##############################################

class FPNode:
    def __init__(self, item, count, parent):
        self.item = item          # Имя элемента
        self.count = count        # Счётчик
        self.parent = parent      # Родительский узел
        self.children = {}        # Дочерние узлы: {item: FPNode}
        self.node_link = None     # Ссылка для объединения узлов с одинаковым именем

    def increment(self, count):
        self.count += count
def update_tree(items, node, header_table, count):
    """
    Рекурсивно обновляет FP-дерево.
    items: отсортированный список элементов транзакции.
    node: текущий узел FP-дерева.
    header_table: таблица заголовков, где для каждого элемента хранится [support, node_link].
    count: количество повторений транзакции.
    """
    if len(items) == 0:
        return
    first_item = items[0]
    if first_item in node.children:
        node.children[first_item].increment(count)
    else:
        new_node = FPNode(first_item, count, node)
        node.children[first_item] = new_node
        if header_table[first_item][1] is None:
            header_table[first_item][1] = new_node
        else:
            current = header_table[first_item][1]
            while current.node_link is not None:
                current = current.node_link
            current.node_link = new_node
    update_tree(items[1:], node.children[first_item], header_table, count)

def create_fp_tree(transactions, min_support):
    """
    Строит FP-дерево из списка транзакций.
    transactions: список транзакций (каждая транзакция – список элементов).
    min_support: абсолютный порог поддержки (число транзакций).
    Возвращает: (root FPNode, header_table).
    """
    header_table = {}
    # Первый проход: подсчет поддержки каждого айтема.
    for trans in transactions:
        for item in trans:
            header_table[item] = header_table.get(item, 0) + 1
    header_table = {item: [support, None] for item, support in header_table.items() if support >= min_support}
    freq_items = set(header_table.keys())
    if len(freq_items) == 0:
        return None, None
    root = FPNode('Null', 1, None)
    # Второй проход: построение FP-дерева.
    for trans in transactions:
        trans_items = [item for item in trans if item in freq_items]
        trans_items.sort(key=lambda item: header_table[item][0], reverse=True)
        if len(trans_items) > 0:
            update_tree(trans_items, root, header_table, 1)
    return root, header_table

def find_prefix_paths(base_item, header_table):
    """
    Находит условную базу паттернов для base_item.
    Возвращает: словарь {path: count}.
    """
    paths = {}
    node = header_table[base_item][1]
    while node is not None:
        prefix_path = []
        parent = node.parent
        while parent is not None and parent.item != 'Null':
            prefix_path.append(parent.item)
            parent = parent.parent
        if len(prefix_path) > 0:
            paths[frozenset(prefix_path)] = paths.get(frozenset(prefix_path), 0) + node.count
        node = node.node_link
    return paths

def create_conditional_fp_tree(prefix_paths, min_support):
    
    #Создает условное FP-дерево на основе агрегированных prefix_paths.
    #prefix_paths: словарь {frozenset(path): count}.
    #min_support: порог поддержки.
    #Возвращает: (conditional_tree, conditional_header).

    freq = {}
    for path, count in prefix_paths.items():
        for item in path:
            freq[item] = freq.get(item, 0) + count
    freq = {item: sup for item, sup in freq.items() if sup >= min_support}
    if len(freq) == 0:
        return None, None
    conditional_header = {item: [sup, None] for item, sup in freq.items()}
    root = FPNode('Null', 1, None)
    for path, count in prefix_paths.items():
        filtered_items = [item for item in path if item in freq]
        filtered_items.sort(key=lambda item: freq[item], reverse=True)
        if len(filtered_items) > 0:
            update_tree(filtered_items, root, conditional_header, count)
    return root, conditional_header     
def mine_fp_tree(fp_tree, header_table, min_support, pre_fix, freq_itemsets):

    #Рекурсивно добывает частые наборы из FP-дерева.
    #pre_fix: текущий префикс (множество).
    #freq_itemsets: словарь, куда записываются найденные частые наборы с их поддержкой.

        sorted_items = sorted(header_table.items(), key=lambda p: p[1][0])
        for base_item, item_info in sorted_items:
            new_freq_set = pre_fix.copy()
            new_freq_set.add(base_item)
            freq_itemsets[frozenset(new_freq_set)] = item_info[0]
            prefix_paths = find_prefix_paths(base_item, header_table)
            conditional_tree, conditional_header = create_conditional_fp_tree(prefix_paths, min_support)
            if conditional_header is not None:
                mine_fp_tree(conditional_tree, conditional_header, min_support, new_freq_set, freq_itemsets)

##############################################
# 4. Генерация ассоциативных правил (A -> Label)
##############################################

def powerset(s):
    """
    Возвращает все непустые подмножества множества s (кроме полного множества).
    """
    s = list(s)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def generate_association_rules(freq_itemsets, min_conf):
    """
    Генерирует ассоциативные правила вида A -> B, где B содержит только элементы, начинающиеся с "Label:".
    freq_itemsets: словарь {frozenset(itemset): support}.
    min_conf: минимальное значение уверенности.
    Возвращает список правил: каждый элемент – кортеж (A, B, support, confidence).
    """
    rules = []
    for itemset in freq_itemsets:
        if len(itemset) < 2:
            continue
        for subset in powerset(itemset):
            A = frozenset(subset)
            B = itemset.difference(A)
            if len(B) > 0 and all(item.startswith("Label:") for item in B):
                support_itemset = freq_itemsets[itemset]
                support_A = freq_itemsets.get(A, 0)
                if support_A > 0:
                    confidence = support_itemset / support_A
                    if confidence >= min_conf:
                        rules.append((A, B, support_itemset, confidence))
    return rules
##############################################
# 5. Основной блок: чтение данных, предобработка, построение транзакций и запуск алгоритма
##############################################
def main():
    # Чтение данных (пример для одного файла CICIDS2017)
    filename = r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\ML&TEST\merged_sampled.csv'  # Замените на путь к вашему файлу
    df = pd.read_csv(filename, low_memory=False)
    df.columns = df.columns.str.strip()
    
    # Отбор признаков. Пример: Protocol, Flow Duration, Flow Bytes/s, Destination Port, Source Port, Label.
    selected_columns = [ 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'Active Mean', 'Idle Mean', 'Label']
    
    # Предобработка: дискретизация и преобразование значений
    df_proc = preprocess_data(df, selected_columns)
    
    # Преобразование в транзакции (каждая строка – список айтемов)
    transactions = transactions_from_df(df_proc)
    print(f"Общее число транзакций: {len(transactions)}")
    
    # Построение FP-дерева
    min_support = 10  # Порог поддержки (абсолютное число транзакций)
    fp_tree, header_table = create_fp_tree(transactions, min_support)
    if fp_tree is None:
        print("Нет частых элементов, удовлетворяющих min_support.")
        return
    
    # Добыча частых наборов (FP-growth)
    freq_itemsets = {}
    mine_fp_tree(fp_tree, header_table, min_support, set(), freq_itemsets)
    print(f"Найдено {len(freq_itemsets)} частых наборов")
    
    # Сохранение частых наборов в CSV файл
    freq_data = []
    for itemset, support in freq_itemsets.items():
        freq_data.append({
            'itemset': ','.join(sorted(itemset)),
            'support': support
        })
    freq_df = pd.DataFrame(freq_data)
    freq_df.to_csv(r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\diplom-project\diplom\result\Deep\frequent_itemsets.csv', index=False)
    print("Частые наборы сохранены в frequent_itemsets.csv")
    
    # Генерация ассоциативных правил вида (A) -> Label
    min_conf = 0.8  # Порог уверенности
    rules = generate_association_rules(freq_itemsets, min_conf)
    print(f"Найдено {len(rules)} ассоциативных правил с min_conf = {min_conf}")
    
    # Вывод всех найденных правил
    rules_data = []
    for A, B, support, conf in rules:
        rules_data.append({
            'A': ','.join(sorted(A)),
            'B': ','.join(sorted(B)),
            'support': support,
            'confidence': conf
        })
    rules_df.to_csv(r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\diplom-project\diplom\result\Deep\association_rules.csv', index=False)
    print("Ассоциативные правила сохранены в association_rules.csv")
    
    # Вывод всех правил в консоль
    for rule in rules_data:
        print(rule)

main()

Общее число транзакций: 56614
Найдено 1340028 частых наборов
Частые наборы сохранены в frequent_itemsets.csv
Найдено 616624 ассоциативных правил с min_conf = 0.8


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

